In [ ]:
import os

from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

print("OPENAI_API_KEY:", os.getenv("OPENAI_API_KEY"))
print("GOOGLE_API_KEY:", os.getenv("GOOGLE_API_KEY"))

In [ ]:
from utils import get_google_api_client

google_api_client = get_google_api_client()

In [48]:
from googleapiclient.discovery import build
from pathlib import Path
from youtube_transcript_api import YouTubeTranscriptApi


def get_latest_video_id(channel_id):
    youtube = build("youtube", "v3", developerKey=os.getenv("GOOGLE_API_KEY"))
    request = youtube.search().list(
        part="snippet", channelId=channel_id, order="date", maxResults=1
    )
    response = request.execute()

    print("deonna response ", response)
    latest_video = response["items"][0]
    video_id = latest_video["id"]["videoId"]
    # video_id = "ghDaTYbmC9o"  # TODO(dnh): temporary hack
    video_title = latest_video["snippet"]["title"]

    return video_id, video_title


# TODO(dnh): Try RAG for this instead
def get_video_transcript(video_id, max_length=4000):
    transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
    transcript = transcript_list.find_transcript(["en"]).fetch()

    total_length = sum(segment["duration"] for segment in transcript)
    middle_time = total_length / 2

    condensed_transcript = ""
    for segment in transcript:
        if segment["start"] >= middle_time:
            text = segment["text"]
            if len(condensed_transcript) + len(text) <= max_length:
                condensed_transcript += " " + text
            else:
                break

    return condensed_transcript.strip()

In [49]:
from langchain.agents import Tool

# Define custom tools
get_latest_video_tool = Tool(
    name="Get Latest YouTube Video",
    func=get_latest_video_id,
    description="Fetches the ID and title of the latest video on a YouTube channel specified by a YouTube channel ID.",
)

get_transcript_tool = Tool(
    name="Get YouTube Channel Transcript by Video ID",
    func=get_video_transcript,
    description="Retrieves the transcript of a given YouTube video from the video id.",
)

tools = [
    get_latest_video_tool,
    get_transcript_tool,
]

In [50]:
from langchain_openai import OpenAI

# Initialize LLM
llm = OpenAI(api_key=os.getenv("OPENAI_API_KEY"), temperature=0.7)

In [51]:
from langchain.agents import initialize_agent

# Initialize the autonomous agent
agent = initialize_agent(
    tools=tools, llm=llm, agent="zero-shot-react-description", verbose=True
)

In [52]:
# High-level prompt for the agent
prompt = "Find the latest episode from the channel with channel ID UCwzCMiicL-hBUzyjWiJaseg and provide a transcript."

# Run the autonomous agent
result = agent.run(prompt)

# Display the response
print(result)



> Entering new AgentExecutor chain...
 First, we need to get the latest video ID from the channel.
Action: Get Latest YouTube Video
Action Input: UCwzCMiicL-hBUzyjWiJasegdeonna response  {'kind': 'youtube#searchListResponse', 'etag': 'y7LP0-yao6yVvovcX1chjmowRS4', 'nextPageToken': 'CAEQAA', 'regionCode': 'US', 'pageInfo': {'totalResults': 597, 'resultsPerPage': 1}, 'items': [{'kind': 'youtube#searchResult', 'etag': '3ajFkHVsD59nx5Na361PRypqTQ0', 'id': {'kind': 'youtube#video', 'videoId': 'ghDaTYbmC9o'}, 'snippet': {'publishedAt': '2024-05-21T01:00:06Z', 'channelId': 'UCwzCMiicL-hBUzyjWiJaseg', 'title': 'KT #664 - KAM PATTERSON + WILLIAM MONTGOMERY + CASEY ROCKET', 'description': 'William Montgomery, Casey Rocket, Kam Patterson, Hans Kim, D Madness, Michael A. Gonzales, Jon Deas, Matthew Muehling, ...', 'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/ghDaTYbmC9o/default.jpg', 'width': 120, 'height': 90}, 'medium': {'url': 'https://i.ytimg.com/vi/ghDaTYbmC9o/mqdefault.jpg', '

In [53]:
from youtube_transcript_api import YouTubeTranscriptApi

video_id = "nwXSS0wBsFA"
transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
transcript = transcript_list.find_transcript(["en"]).fetch()
print(transcript)

[{'text': "hey y'all the new kill merch store is up", 'start': 0.12, 'duration': 5.34}, {'text': 'and running at a thousand miles an hour', 'start': 3.419, 'duration': 6.021}, {'text': 'it is our official kill Tony online Shop', 'start': 5.46, 'duration': 7.74}, {'text': 'killmerch.com t-shirts hoodies Bonsai', 'start': 9.44, 'duration': 6.159}, {'text': 'Made Real Texas Leather joke books kill', 'start': 13.2, 'duration': 5.64}, {'text': 'Tony hats kill Tony knit winter hats', 'start': 15.599, 'duration': 6.481}, {'text': 'beer koozies bandanas trucker hats', 'start': 18.84, 'duration': 7.38}, {'text': 'mcvader made posters some Ryan J E belt', 'start': 22.08, 'duration': 6.42}, {'text': 'art coming in soon and kill Tony', 'start': 26.22, 'duration': 4.139}, {'text': 'stickers come with a lot of the', 'start': 28.5, 'duration': 4.14}, {'text': 'purchases that you make I personally', 'start': 30.359, 'duration': 4.741}, {'text': 'love the kill Tony NASCAR shirt the kill', 'start': 32.6